In [16]:
from azure.cosmos import CosmosClient, exceptions, PartitionKey
import json
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv
import os
#load_dotenv()
# Define your Cosmos DB account information
endpoint = "https://anildwacosmoswestus.documents.azure.com:443/"

credential = DefaultAzureCredential()
# Initialize the Cosmos client
client = CosmosClient(endpoint, credential=credential)

In [2]:
database = client.create_database(id="sparkdb")

In [4]:
for db in client.list_databases():
    print(db)

{'id': 'sparkdb', '_rid': '5WwTAA==', '_self': 'dbs/5WwTAA==/', '_etag': '"00002402-0000-0700-0000-681fe21b0000"', '_colls': 'colls/', '_users': 'users/', '_ts': 1746919963}
{'id': 'booksdb', '_rid': 'adFqAA==', '_self': 'dbs/adFqAA==/', '_etag': '"00008a00-0000-0700-0000-680936200000"', '_colls': 'colls/', '_users': 'users/', '_ts': 1745434144}


In [22]:
import requests


search_token = credential.get_token("https://search.azure.com/.default").token
print(search_token)
azure_search_endpoint = "https://anildwaaisearch-basic.search.windows.net"
def create_simple_index(index_name: str, analyzer_name: str = "en.microsoft", language_suffix: str = "en"):
    index_schema = {
        "name": index_name,
        "fields": [
            {
                "name": "id",
                "type": "Edm.String",
                "key": True,
                "sortable": True,
                "filterable": True,
                "facetable": True
            },
            # Existing fields
            # Adding the new fields as searchable text fields
            {
                "name": "content",
                "type": "Edm.String",
                "searchable": True
            },
            
            # Vector fields for embeddings
            {
                "name": "contentVector",
                "type": "Collection(Edm.Single)",
                "searchable": True,
                "dimensions": 1536,
                "vectorSearchProfile": "amlHnswProfile"
                
            }
            # Existing fields such as lastUpdated, suggesters, scoringProfiles, etc.
        ],
        "scoringProfiles": [
            
        ],
        "suggesters": [
            
        ],
        "vectorSearch": {
                "algorithms": [
                    {
                        "name": "amlHnsw",
                        "kind": "hnsw",
                        "hnswParameters": {
                        "m": 4,
                        "metric": "cosine"
                        }
                    }
                
                ],
                "profiles": [
                    {
                        "name": "amlHnswProfile",
                        "algorithm": "amlHnsw",
                        "vectorizer": None
                    }
                
                ], 
                "vectorizers": []
                
        },
            "semantic": {
                "configurations": [
                    {
                        "name": "aml-semantic-config",
                        "prioritizedFields": {
                            "titleField": {
                                "fieldName": "content"
                            },
                            "prioritizedKeywordsFields": [
                                {
                                    "fieldName": "content"
                                }
                            
                            ],
                            "prioritizedContentFields": [
                                {
                                    "fieldName": f"content"
                                }
                            ]
                        }
                    }
                ]
            }
        }



    headers = {'Content-Type': 'application/json',
            'Authorization': f'Bearer {search_token}'  }

    print(headers)
    # Create Index
    url = azure_search_endpoint + "/indexes/" + index_name + "?api-version=2024-07-01"
    print(url)
    response  = requests.put(url, headers=headers, json=index_schema)
    print(response.text)
        #response = requests.get(url, headers=headers)
        #if response.status_code == 404:
        #    response  = requests.put(url, headers=headers, json=index_schema)
        #    index = response.json()
        #    print(index)
        #else:
        #    print(response)
        #    print("Index already exists")


create_simple_index("test_index")


eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IkNOdjBPSTNSd3FsSEZFVm5hb01Bc2hDSDJYRSIsImtpZCI6IkNOdjBPSTNSd3FsSEZFVm5hb01Bc2hDSDJYRSJ9.eyJhdWQiOiJodHRwczovL3NlYXJjaC5henVyZS5jb20iLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC8xNTAzMDViMy1jYzRiLTQ2ZGQtOTkxMi00MjU2NzhkYjE0OTgvIiwiaWF0IjoxNzQ2OTQ0OTI3LCJuYmYiOjE3NDY5NDQ5MjcsImV4cCI6MTc0Njk0ODgyNywiYWlvIjoiazJSZ1lFZzZreHg4cmVUWjc4WW0vaHBwSi9rc0FBPT0iLCJhcHBpZCI6Ijc5OWVmOTNlLTFiNGMtNGJiMS04OTMwLTVkOTNkYWExM2ZmMyIsImFwcGlkYWNyIjoiMSIsImlkcCI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzE1MDMwNWIzLWNjNGItNDZkZC05OTEyLTQyNTY3OGRiMTQ5OC8iLCJpZHR5cCI6ImFwcCIsIm9pZCI6ImVmY2EyNTI1LTQ3ZWUtNDdiZC1hM2Q3LTM0Mjg4MzI0MWNmMiIsInJoIjoiMS5BWEVCc3dVREZVdk0zVWFaRWtKV2VOc1VtSUNqRFloZW1KaEJnYm5nV3h6Rk1WaHhBUUJ4QVEuIiwic3ViIjoiZWZjYTI1MjUtNDdlZS00N2JkLWEzZDctMzQyODgzMjQxY2YyIiwidGlkIjoiMTUwMzA1YjMtY2M0Yi00NmRkLTk5MTItNDI1Njc4ZGIxNDk4IiwidXRpIjoiX1FkeU5lc1dwa0tkRV9KNG5rUkJBQSIsInZlciI6IjEuMCIsInhtc19pZHJlbCI6IjMwIDcifQ.JeTSS2300x7yYVVnUB_ZDwdfbOD4cDEppT2Of04EGGp31S4z40zB5Ji69WA5